In [61]:
# Import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.firefox import GeckoDriverManager
from selenium.common.exceptions import StaleElementReferenceException

In [2]:
# Set up splinter
executable_path = {'executable_path': GeckoDriverManager().install()}
browser = Browser('firefox', **executable_path, headless=False)



====== WebDriver manager ======
Current firefox version is 103.0
Get LATEST geckodriver version for 103.0 firefox
Driver [/Users/zcvalis/.wdm/drivers/geckodriver/macos/v0.31.0/geckodriver] found in cache


In [140]:
# Visit Formula 1 site
url = 'https://www.formula1.com/en/results.html'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.resultsarchive-filter-wrap', wait_time=5)

True

In [141]:
# Set up html parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.resultsarchive-filter-container')

In [110]:
# Click accept cookies
browser.find_by_id('truste-consent-button').click()

ElementDoesNotExist: no elements could be found with id "truste-consent-button"

In [142]:
# Get list of partial race links
partial_link = slide_elem.findAll('a', class_='resultsarchive-filter-item-link FilterTrigger')
race_partial_hrefs = [link.get('href') for link in race_result_partial_link[-22:-9]]

In [114]:
for count, value in enumerate(race_partial_hrefs):
    print(f"Count: {count}")
    print(f"Value: {value}")

Count: 0
Value: /en/results.html/2022/races/1124/bahrain/race-result.html
Count: 1
Value: /en/results.html/2022/races/1125/saudi-arabia/race-result.html
Count: 2
Value: /en/results.html/2022/races/1108/australia/race-result.html
Count: 3
Value: /en/results.html/2022/races/1109/italy/race-result.html
Count: 4
Value: /en/results.html/2022/races/1110/miami/race-result.html
Count: 5
Value: /en/results.html/2022/races/1111/spain/race-result.html
Count: 6
Value: /en/results.html/2022/races/1112/monaco/race-result.html
Count: 7
Value: /en/results.html/2022/races/1126/azerbaijan/race-result.html
Count: 8
Value: /en/results.html/2022/races/1113/canada/race-result.html
Count: 9
Value: /en/results.html/2022/races/1114/great-britain/race-result.html
Count: 10
Value: /en/results.html/2022/races/1115/austria/race-result.html
Count: 11
Value: /en/results.html/2022/races/1116/france/race-result.html
Count: 12
Value: /en/results.html/2022/races/1117/hungary/race-result.html


In [143]:
# Function adapted to Python from http://darrellgrainger.blogspot.com/2012/06/staleelementexception.html
def retryingFindClick(link):
    result = False
    attempts = 0
    while (attempts < 2):
        try:
            browser.links.find_by_partial_href(link).click()
            result = True
            break
        except StaleElementReferenceException:
            attempts += 1
    
    return result

In [144]:
# Index in resultsarchive-filter-item-link FilterTrigger html class
f1_url = 'https://www.formula1.com'
data = pd.DataFrame()
df = pd.DataFrame()

for count, race_href in enumerate(race_partial_hrefs):
    print(f"{count} iterate")
    print(f"race link | {race_href}")
    
    # Click link using partial
    retryingFindClick(race_href)
    
    # Get full race results link
    race_result_full_url = f'{f1_url}{race_href}'
    print(race_result_full_url)
    data = pd.read_html(race_result_full_url)[0]
    
    # Delay for loading the page
    browser.is_element_present_by_css('a.side-nav-item-link ArchiveLink')
    
    # Get side column data
    # Set up new html parser and get partial link
    html = browser.html
    news_soup = soup(html, 'html.parser')
    slide_elem = news_soup.select_one('div.resultsarchive-wrapper')
    
    # Delay for loading the page
    browser.is_element_present_by_css('ul.resultsarchive-side-nav')
    
    side_col_data = slide_elem.findAll('a', class_='side-nav-item-link ArchiveLink')
    # Loop through side column data
    for i in range(len(side_col_data)):
        side_col_data_partial_link = side_col_data[i].get('href')
        print(side_col_data_partial_link)
        side_col_data_full_url = f'{f1_url}{side_col_data_partial_link}'
        data = pd.read_html(side_col_data_full_url)[0]
    

    # Reset html parser
    html = browser.html
    news_soup = soup(html, 'html.parser')
    slide_elem = news_soup.select_one('div.resultsarchive-filter-item-link FilterTrigger')

0 iterate
race link | /en/results.html/2022/races/1124/bahrain/race-result.html
https://www.formula1.com/en/results.html/2022/races/1124/bahrain/race-result.html
/en/results.html/2022/races/1124/bahrain/fastest-laps.html
/en/results.html/2022/races/1124/bahrain/pit-stop-summary.html
/en/results.html/2022/races/1124/bahrain/starting-grid.html
/en/results.html/2022/races/1124/bahrain/qualifying.html
/en/results.html/2022/races/1124/bahrain/practice-3.html
/en/results.html/2022/races/1124/bahrain/practice-2.html
/en/results.html/2022/races/1124/bahrain/practice-1.html
1 iterate
race link | /en/results.html/2022/races/1125/saudi-arabia/race-result.html
https://www.formula1.com/en/results.html/2022/races/1125/saudi-arabia/race-result.html
/en/results.html/2022/races/1125/saudi-arabia/fastest-laps.html
/en/results.html/2022/races/1125/saudi-arabia/pit-stop-summary.html
/en/results.html/2022/races/1125/saudi-arabia/starting-grid.html
/en/results.html/2022/races/1125/saudi-arabia/qualifying.ht

In [145]:
data

,Unnamed: 0,Pos,No,Driver,Car,Time,Gap,Laps,Unnamed: 8
0,NaN,1,55,Carlos Sainz SAI,Ferrari,1:18.750,NaN,26,NaN
1,NaN,2,1,Max Verstappen VER,Red Bull Racing RBPT,1:18.880,+0.130s,24,NaN
2,NaN,3,16,Charles Leclerc LEC,Ferrari,1:19.039,+0.289s,26,NaN
3,NaN,4,4,Lando Norris NOR,McLaren Mercedes,1:19.299,+0.549s,21,NaN
4,NaN,5,63,George Russell RUS,Mercedes,1:19.606,+0.856s,28,NaN
5,NaN,6,11,Sergio Perez PER,Red Bull Racing RBPT,1:19.622,+0.872s,24,NaN
6,NaN,7,44,Lewis Hamilton HAM,Mercedes,1:19.710,+0.960s,25,NaN
7,NaN,8,3,Daniel Ricciardo RIC,McLaren Mercedes,1:19.841,+1.091s,28,NaN
8,NaN,9,31,Esteban Ocon OCO,Alpine Renault,1:20.348,+1.598s,30,NaN
9,NaN,10,14,Fernando Alonso ALO,Alpine Renault,1:20.377,+1.627s,27,NaN


In [139]:
data.to_excel("output.xlsx")